# Notebook 1: Carga de Datos

## Objetivo
Este notebook se encarga exclusivamente de descargar y almacenar los datos necesarios para toda la práctica.

## Estructura
1. Imports y Configuración
2. Verificación del Archivo desde Google Drive
3. Carga del Dataset
4. Separación del Benchmark
5. Guardado del Dataset Completo

## 1. Imports y Configuración

In [29]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import yfinance as yf

# Configuración de pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# Constantes
DATA_RAW_DIR = '../datos/raw'
BENCHMARK_TICKER = 'SPY'
START_DATE = '2015-01-01'
PARQUET_NAME = 'sp500_history.parquet'

print('Librerías importadas correctamente')

Librerías importadas correctamente


## 2. Verificación del Archivo desde Google Drive

**Importante:** El archivo debe descargarse manualmente desde Google Drive y colocarse en la carpeta `datos/raw/` con el nombre `historical_prices.parquet`.

Enlace: https://drive.google.com/file/d/1nvubXdAu0EONlrP_yrURZbnPhBQ-uDaB/view?usp=sharing

In [14]:
def find_parquet(filename):
    """
    Busca el archivo parquet en diferentes ubicaciones posibles.
    
    Parámetros:
    -----------
    filename : str
        Nombre del archivo a buscar
    
    Retorna:
    --------
    str o None
        Ruta al archivo si se encuentra, None en caso contrario
    """
    # Lista de rutas posibles a probar
    possible_paths = [
        f'../datos/raw/{filename}',
        f'datos/raw/{filename}',
        f'../{filename}',
        filename,
        f'../../datos/raw/{filename}',
        f'../../../datos/raw/{filename}',
    ]
    
    for path in possible_paths:
        try:
            # Intentar abrir el archivo con pyarrow directamente
            parquet_file = pq.ParquetFile(path)
            parquet_file.close()
            return path
        except FileNotFoundError:
            continue
        except Exception:
            # Si hay otro error (no FileNotFoundError), el archivo existe
            # pero hay un problema al leerlo, lo intentamos de todas formas
            return path
    
    return None


# Resolver ruta del Parquet de forma robusta
parquet_path = find_parquet(PARQUET_NAME)

if parquet_path is None:
    raise FileNotFoundError(
        f'No se encuentra el archivo: {PARQUET_NAME}. '
        f'Asegúrate de estar dentro de la carpeta del proyecto o sube el parquet.'
    )

downloaded_file_path = parquet_path
print(f'Archivo encontrado en: {downloaded_file_path}')

Archivo encontrado en: ../datos/raw/sp500_history.parquet


## 3. Carga del Dataset

In [20]:
def load_parquet_dataset(file_path):
    """
    Carga un dataset desde un archivo Parquet usando pyarrow directamente.
    
    Parámetros:
    -----------
    file_path : str
        Ruta al archivo Parquet
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con los datos cargados
    """
    print(f'Cargando dataset desde: {file_path}')
    try:
        # Usar pyarrow directamente para evitar conflictos de tipos
        table = pq.read_table(file_path)
        df = table.to_pandas()
        print('Dataset cargado correctamente')
        return df
    except Exception as e:
        print(f'Error al cargar el archivo con pyarrow: {str(e)}')
        print('Intentando con pandas directamente...')
        try:
            # Si falla pyarrow, intentar con pandas
            df = pd.read_parquet(file_path, engine='pyarrow')
            print('Dataset cargado correctamente')
            return df
        except Exception as e2:
            print(f'Error al cargar con pandas: {str(e2)}')
            raise


# Cargar el dataset completo
dataset = load_parquet_dataset(downloaded_file_path)

# Verificar estructura básica
print('\n=== INFORMACIÓN DEL DATASET ===')
print(f'Shape del dataset: {dataset.shape}')
print(f'Columnas: {len(dataset.columns)}')
print(f'Filas: {len(dataset)}')
print(f'Columnas disponibles: {list(dataset.columns)}')

# Si el dataset tiene columna 'date', convertirla a datetime y usarla como índice
if 'date' in dataset.columns:
    print('\nConvirtiendo columna date a datetime...')
    dataset['date'] = pd.to_datetime(dataset['date'], errors='coerce')
    dataset = dataset.set_index('date')
    dataset = dataset.sort_index()
elif not isinstance(dataset.index, pd.DatetimeIndex):
    print('Convirtiendo índice a DatetimeIndex...')
    dataset.index = pd.to_datetime(dataset.index, errors='coerce')
    dataset = dataset.sort_index()

# Mostrar rango de fechas
print(f'\nRango de fechas disponible:')
print(f'  Fecha inicio: {dataset.index.min()}')
print(f'  Fecha fin: {dataset.index.max()}')

# Contar tickers únicos
if 'symbol' in dataset.columns:
    tickers = sorted(dataset['symbol'].unique())
    print(f'\nNúmero de tickers únicos: {len(tickers)}')
    print(f'Tickers disponibles: {tickers}')
elif isinstance(dataset.columns, pd.MultiIndex):
    tickers = sorted(dataset.columns.get_level_values(0).unique())
    print(f'\nNúmero de tickers únicos: {len(tickers)}')
    print(f'Tickers disponibles: {tickers}')
else:
    tickers = sorted(dataset.columns.unique())
    print(f'\nNúmero de columnas: {len(tickers)}')
    print(f'Columnas disponibles: {tickers}')

# Mostrar información detallada del dataset
print('\n=== INFORMACIÓN DETALLADA DEL DATASET ===')
print(f'Total de registros: {len(dataset):,}')
print(f'Total de columnas: {len(dataset.columns)}')
print(f'\nTipos de datos por columna:')
print(dataset.dtypes)

# Mostrar primeras filas para verificación
print('\n=== PRIMERAS 10 FILAS DEL DATASET ===')
with pd.option_context('display.max_columns', None, 
                       'display.max_rows', 10,
                       'display.width', None,
                       'display.max_colwidth', 50):
    print(dataset.head(10))

# Mostrar últimas filas
print('\n=== ÚLTIMAS 10 FILAS DEL DATASET ===')
with pd.option_context('display.max_columns', None, 
                       'display.max_rows', 10,
                       'display.width', None,
                       'display.max_colwidth', 50):
    print(dataset.tail(10))



Cargando dataset desde: ../datos/raw/sp500_history.parquet
Dataset cargado correctamente

=== INFORMACIÓN DEL DATASET ===
Shape del dataset: (7250110, 14)
Columnas: 14
Filas: 7250110
Columnas disponibles: ['date', 'symbol', 'assetid', 'security_name', 'sector', 'industry', 'subsector', 'in_sp500', 'open', 'high', 'low', 'close', 'volume', 'unadjusted_close']

Convirtiendo columna date a datetime...

Rango de fechas disponible:
  Fecha inicio: 1990-01-02 00:00:00
  Fecha fin: 2026-01-30 00:00:00

Número de tickers únicos: 1289
Tickers disponibles: ['A', 'AABA-201910', 'AAL', 'AAL-199702', 'AAMRQ-201312', 'AAP', 'AAPL', 'AAV-199901', 'ABBV', 'ABI-200811', 'ABKFQ-201304', 'ABMD-202212', 'ABNB', 'ABS-200606', 'ABT', 'ACAS-201701', 'ACGL', 'ACKH-200712', 'ACN', 'ACS-201002', 'ACV-201105', 'ACY-199411', 'ADBE', 'ADCT-201012', 'ADI', 'ADM', 'ADP', 'ADSK', 'ADT-201604', 'AEE', 'AEP', 'AES', 'AET-201811', 'AFL', 'AFS-200011', 'AGC-200108', 'AGN-201503', 'AGN-202005', 'AHM-199810', 'AIG', 'AIT-1

## 4. Separación del Benchmark

In [33]:
def download_benchmark_data(benchmark_ticker, start_date):
    """
    Descarga los datos del benchmark desde yfinance y los formatea
    con la misma estructura que el dataset del parquet.
    
    Parámetros:
    -----------
    benchmark_ticker : str
        Símbolo del ticker del benchmark (ej: 'SPY')
    start_date : str
        Fecha de inicio en formato 'YYYY-MM-DD'
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con datos del benchmark con las mismas columnas que el parquet
    """
    print(f'Descargando datos de {benchmark_ticker} desde {start_date}...')
    
    try:
        ticker = yf.Ticker(benchmark_ticker)
        hist = ticker.history(start=start_date)
        
        if hist.empty:
            raise ValueError(f'No se pudieron descargar datos de {benchmark_ticker}')
        
        # Obtener información del ticker
        info = ticker.info
        
        # Formatear datos con la misma estructura que el parquet
        benchmark_data = pd.DataFrame({
            'symbol': benchmark_ticker,
            'assetid': 0,  # No disponible en yfinance
            'security_name': info.get('longName', benchmark_ticker),
            'sector': info.get('sector', 'Unknown'),
            'industry': info.get('industry', 'Unknown'),
            'subsector': info.get('industryDisp', 'Unknown'),
            'in_sp500': 1,  # SPY es el ETF del S&P 500
            'open': hist['Open'].values,
            'high': hist['High'].values,
            'low': hist['Low'].values,
            'close': hist['Close'].values,
            'volume': hist['Volume'].values,
            'unadjusted_close': hist['Close'].values  # yfinance no proporciona unadjusted
        })
        
        # Usar la fecha como índice
        benchmark_data.index = hist.index
        benchmark_data.index.name = 'date'
        
        # Ordenar por fecha
        benchmark_data = benchmark_data.sort_index()
        
        print(f'Benchmark {benchmark_ticker} descargado correctamente')
        print(f'Shape del benchmark: {benchmark_data.shape}')
        print(f'Rango de fechas: {benchmark_data.index.min()} a {benchmark_data.index.max()}')
        print(f'Columnas: {list(benchmark_data.columns)}')
        
        return benchmark_data
        
    except Exception as e:
        print(f'Error al descargar {benchmark_ticker}: {str(e)}')
        raise


# Descargar datos del benchmark SPY
spy_data = download_benchmark_data(BENCHMARK_TICKER, START_DATE)

# Mostrar información detallada del benchmark
print('\n=== INFORMACIÓN DETALLADA DEL BENCHMARK ===')
print(f'Total de registros: {len(spy_data):,}')
print(f'Total de columnas: {len(spy_data.columns)}')
print(f'Número de días: {len(spy_data)}')
print(f'\nTipos de datos por columna:')
print(spy_data.dtypes)

# Estadísticas básicas de precios
print('\n=== ESTADÍSTICAS DE PRECIOS ===')
price_cols = ['open', 'high', 'low', 'close']
with pd.option_context('display.max_columns', None):
    print(spy_data[price_cols].describe())

# Mostrar primeras filas
print('\n=== PRIMERAS 10 FILAS DEL BENCHMARK ===')
with pd.option_context('display.max_columns', None, 
                       'display.max_rows', 10,
                       'display.width', None,
                       'display.max_colwidth', 50):
    print(spy_data.head(10))

# Mostrar últimas filas
print('\n=== ÚLTIMAS 10 FILAS DEL BENCHMARK ===')
with pd.option_context('display.max_columns', None, 
                       'display.max_rows', 10,
                       'display.width', None,
                       'display.max_colwidth', 50):
    print(spy_data.tail(10))

# Guardar benchmark usando pyarrow directamente
spy_output_path = f'{DATA_RAW_DIR}/spy_data.parquet'
try:
    # Convertir DataFrame a tabla de pyarrow y guardar
    table = pa.Table.from_pandas(spy_data)
    pq.write_table(table, spy_output_path)
    print(f'\nBenchmark guardado en: {spy_output_path}')
except (OSError, FileNotFoundError):
    print(f'Error: No se pudo crear el directorio. '
          f'Asegúrate de que {DATA_RAW_DIR} existe.')
    raise
except Exception as e:
    print(f'Error al guardar el benchmark: {str(e)}')
    raise

Descargando datos de SPY desde 2015-01-01...
Benchmark SPY descargado correctamente
Shape del benchmark: (2796, 13)
Rango de fechas: 2015-01-02 00:00:00-05:00 a 2026-02-13 00:00:00-05:00
Columnas: ['symbol', 'assetid', 'security_name', 'sector', 'industry', 'subsector', 'in_sp500', 'open', 'high', 'low', 'close', 'volume', 'unadjusted_close']

=== INFORMACIÓN DETALLADA DEL BENCHMARK ===
Total de registros: 2,796
Total de columnas: 13
Número de días: 2796

Tipos de datos por columna:
symbol               object
assetid               int64
security_name        object
sector               object
industry             object
subsector            object
in_sp500              int64
open                float64
high                float64
low                 float64
close               float64
volume                int64
unadjusted_close    float64
dtype: object

=== ESTADÍSTICAS DE PRECIOS ===
           open      high       low     close
count 2796.0000 2796.0000 2796.0000 2796.0000
mean   34

## 5. Guardado del Dataset Completo

Guarda el dataset completo procesado (todos los tickers del S&P 500) en `tickers_data.parquet`. Diferencias: `sp500_history.parquet` es el original sin procesar, `spy_data.parquet` contiene solo el benchmark SPY, y `tickers_data.parquet` es el dataset completo procesado y listo para usar.

In [34]:
def save_dataset_parquet(df, output_path):
    """
    Guarda un DataFrame en formato Parquet usando pyarrow directamente.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame a guardar
    output_path : str
        Ruta donde guardar el archivo
    """
    print(f'Guardando dataset en: {output_path}')
    try:
        # Convertir DataFrame a tabla de pyarrow y guardar
        table = pa.Table.from_pandas(df)
        pq.write_table(table, output_path)
        print(f'Dataset guardado exitosamente')
    except Exception as e:
        print(f'Error al guardar el dataset: {str(e)}')
        raise


# Guardar dataset completo de tickers
tickers_output_path = f'{DATA_RAW_DIR}/tickers_data.parquet'
try:
    save_dataset_parquet(dataset, tickers_output_path)
except (OSError, FileNotFoundError):
    print(f'Error: No se pudo crear el directorio. '
          f'Asegúrate de que {DATA_RAW_DIR} existe.')
    raise

# Resumen final
print('\n=== RESUMEN DEL NOTEBOOK 1 ===')
print('Proceso completado exitosamente')
print(f'\nArchivos generados:')
print(f'  1. {downloaded_file_path}')
print(f'  2. {spy_output_path}')
print(f'  3. {tickers_output_path}')
print(f'\nInformación del dataset:')
print(f'  - Total de tickers: {len(tickers)}')
print(f'  - Rango de fechas: {dataset.index.min()} a {dataset.index.max()}')
print(f'  - Total de registros: {len(dataset)}')
print('\nNotebook 1 completado correctamente')

Guardando dataset en: ../datos/raw/tickers_data.parquet
Dataset guardado exitosamente

=== RESUMEN DEL NOTEBOOK 1 ===
Proceso completado exitosamente

Archivos generados:
  1. ../datos/raw/sp500_history.parquet
  2. ../datos/raw/spy_data.parquet
  3. ../datos/raw/tickers_data.parquet

Información del dataset:
  - Total de tickers: 1289
  - Rango de fechas: 1990-01-02 00:00:00 a 2026-01-30 00:00:00
  - Total de registros: 7250110

Notebook 1 completado correctamente
